In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pathlib import Path
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [135]:
map_of_streams_actual1 = {
    'Business Analysis': ['business analyst'], 'Business Intelligence': ['business intelligence', 'data analyst'],
    'Cloud Computing': ['cloud', 'azure'], 'Compliance and Risk': ['compliance and risk', 'risk analyst'],
    'Cyber Security': ['cyber security'], 'Development': ['software developer']}

map_of_streams_actual2 = {
    'Information Security Management': ['information security'], 'IT Service Management': ['technical support']
    , 'Robotic Process Automation': ['rpa'],
    'Testing': ['software tester', 'java test engineer']}

map_of_streams_3 = {
    'PMO': ['project manager']
}
map_of_streams_tester = {
    'Testing': ['software test']
}
# map_of_streams_risk = {
#     'Compliance and Risk': ['risk analyst']
# }
map_of_streams_info_security = {
    'Information Security Management': ['information security']
}
# 


In [136]:
job_titles = []
links = []
streams = []
descriptions = []

def scrape_web_job_description(map_of_streams):
    # specify driver path
    DRIVER_PATH = '/Users/kaykay/Downloads/chromedriver'

    # stop pop-ups
    chrome_options = Options()

    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("start-maximized")
    chrome_options.add_argument("--disable-extensions")

    # Pass the argument 1 to allow and 2 to block
    chrome_options.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications": 1
    })

    driver = webdriver.Chrome(options=chrome_options, executable_path=DRIVER_PATH)
    driver.get('https://www.indeed.co.uk/advanced_search')
    #driver.find_element_by_id("popover-x").click()
    df_stream_description = pd.DataFrame()

    for stream, job_descriptions in map_of_streams.items():
        print(stream, ' >>>>>>>')

        for job_description in job_descriptions:
            print(job_description, '>>>>>')

            # Search in the advanced page
            # Input it also in the "With all these words in the title" box
            try:
                search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
            except:
                print("")
#                 close_popup = driver.find_element_by_id("popover-x")
#                 close_popup.click()
#                 close_cookie_popup = driver.find_element_by_id("onetrust-accept-btn-handler")
#                 close_cookie_popup.click()
            search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
            search_job.send_keys([job_description])
            driver.implicitly_wait(3)
            
            # Input it also in the "With these words in the title" box
            search_job = driver.find_element_by_xpath('//input[@id="as_ttl"]')
            search_job.send_keys([job_description])

            # for rpa look for robotic in the description as well
            if stream == 'Robotic Process Automation':
                search_job = driver.find_element_by_xpath('//input[@id="as_any"]')
                search_job.send_keys(['robotic'])
            
            driver.implicitly_wait(3)
            # set display limit of 30 results per page
            display_limit = driver.find_element_by_xpath('//select[@id="limit"]//option[@value="50"]')
            display_limit.click()
            # sort by date
            sort_option = driver.find_element_by_xpath('//select[@id="sort"]//option[@value="date"]')
            sort_option.click()
            search_button = driver.find_element_by_xpath('//*[@id="fj"]')
            search_button.click()
            try:
                close_cookie_popup = driver.find_element_by_id("onetrust-accept-btn-handler")
                close_cookie_popup.click()
            except:
                print("No cookies popup")

            # Save the data to a table
            # let the driver wait 3 seconds to locate the element before exiting out
            driver.implicitly_wait(3)

            for i in range(0, 40):

                job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')

                for job in job_card:

                    try:

                        title = job.find_element_by_xpath('.//h2[@class="title"]//a').text
                        job_titles.append(title)
                        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
                        streams.append(stream)

                    except:
                        title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
                        job_titles.append(title)
                        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
                        streams.append(stream)

                try:
                    next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i + 2))
                    next_page.click()
#                     WebDriverWait(driver, 3).until(EC.alert_is_present(),
#                                                     'Timed out waiting for PA creation ' +
#                                                     'confirmation popup to appear.')

#                     alert = browser.switch_to.alert
#                     alert.accept()
                except ElementClickInterceptedException:
                    close_popup = driver.find_element_by_id("popover-x")
                    close_popup.click()

                except TimeoutException:
                    print("no alert")

                except NoSuchElementException:
                    try:
                        close_popup = driver.find_element_by_id("popover-x")
                        close_popup.click()

                    except:
                        try:
                            next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
                            next_page.click()
                            driver.implicitly_wait(3)
                        except:
                            break

                except:

                    next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
                    next_page.click()



                print("Page: {}".format(str(i + 2)))

            # we need to be able to clear search and go on to the next role
            # go to the clear advanced search page to insert new job description
            driver.get('https://www.indeed.co.uk/advanced_search')

    for link in links:
        try:
            
            driver.get(link)
            jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
        except NoSuchElementException:
            print(" empty website")
            jd = ""
                
        
        descriptions.append(jd)
    
    #del links[:]

    df_stream_description['Title'] = job_titles
    df_stream_description['Description'] = descriptions
    df_stream_description['Stream'] = streams

    return df_stream_description

In [70]:
df1 = scrape_web_job_description(map_of_streams_actual1)

Business Analysis  >>>>>>>
alert accepted
Page: 2
alert accepted
Page: 3
Page: 4
alert accepted
Page: 5
alert accepted
Page: 6
alert accepted
Page: 7
alert accepted
Page: 8
alert accepted
Page: 9
alert accepted
Page: 10
alert accepted
Page: 11
alert accepted
Page: 12
alert accepted
Page: 13
alert accepted
Page: 14
alert accepted
Page: 15
alert accepted
Page: 16
alert accepted
Page: 17
alert accepted
Page: 18
Business Intelligence  >>>>>>>
No cookies popup
alert accepted
Page: 2
alert accepted
Page: 3
alert accepted
Page: 4
No cookies popup
alert accepted
Page: 2
alert accepted
Page: 3
alert accepted
Page: 4
alert accepted
Page: 5
alert accepted
Page: 6
alert accepted
Page: 7
alert accepted
Page: 8
alert accepted
Page: 9
alert accepted
Page: 10
alert accepted
Page: 11
Cloud Computing  >>>>>>>
No cookies popup
alert accepted
Page: 2
alert accepted
Page: 3
alert accepted
Page: 4
alert accepted
Page: 5
alert accepted
Page: 6
alert accepted
Page: 7
alert accepted
Page: 8
alert accepted
Page

In [71]:

print(df1.tail())
print(df1['Stream'].unique())

                                                  Title  \
1195                      Software Developer III (Ruby)   
1196  Java Developer / Junior Software Engineer MSc PhD   
1197                                 Software Developer   
1198    Senior Software Engineer - Developer Experience   
1199         Lead Developer / Senior Software Developer   

                                            Description       Stream  
1195  We’re Charlie, we build HR software and provid...  Development  
1196  Java Developer / Junior Software Engineer (Mat...  Development  
1197  An opportunity has popped up for a talented So...  Development  
1198  Fastly helps people stay better connected with...  Development  
1199  Our client is a fast-growing innovative compan...  Development  
['Business Analysis' 'Business Intelligence' 'Cloud Computing'
 'Compliance and Risk' 'Cyber Security' 'Development']


In [72]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1200 non-null   object
 1   Description  1200 non-null   object
 2   Stream       1200 non-null   object
dtypes: object(3)
memory usage: 28.2+ KB


In [134]:
len(df1[df1['Stream'] == 'Information Security Management'])

0

In [73]:
import pickle

with open('/Users/kaykay/Downloads/RecognitionEngineProject/recognitionengine/src/ml_determine_stream/df1_jd', 'wb') as fh:  # notice that you need the 'wb' for the dump
    pickle.dump(df1, fh)

In [106]:
df2 = scrape_web_job_description(map_of_streams_actual2)


Information Security Management  >>>>>>>
information security >>>>>
Page: 2
Page: 3
Page: 4
Page: 5
IT Service Management  >>>>>>>
technical support >>>>>
No cookies popup
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Robotic Process Automation  >>>>>>>
rpa >>>>>
No cookies popup
Testing  >>>>>>>
software tester >>>>>
No cookies popup
Page: 2
java test engineer >>>>>
No cookies popup


In [107]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        231 non-null    object
 1   Description  231 non-null    object
 2   Stream       231 non-null    object
dtypes: object(3)
memory usage: 5.5+ KB
None


In [38]:
print(df2['Stream'].unique())

['Information Security Management' 'IT Service Management'
 'Robotic Process Automation' 'Testing']


In [109]:
print(df2[df2['Stream'] == 'IT Service Management'].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 75 to 179
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        105 non-null    object
 1   Description  105 non-null    object
 2   Stream       105 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB
None


In [114]:
with open('/Users/kaykay/Downloads/RecognitionEngineProject/recognitionengine/src/ml_determine_stream/df2_jd', 'wb') as fh:  # notice that you need the 'wb' for the dump
    pickle.dump(df2, fh)

In [58]:
print(df1[df1['Stream'] == 'Development'].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 570 to 659
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        90 non-null     object
 1   Description  90 non-null     object
 2   Stream       90 non-null     object
dtypes: object(3)
memory usage: 2.8+ KB
None


In [119]:
df3 = scrape_web_job_description(map_of_streams_3)

PMO  >>>>>>>
project manager >>>>>
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20


In [120]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        285 non-null    object
 1   Description  285 non-null    object
 2   Stream       285 non-null    object
dtypes: object(3)
memory usage: 6.8+ KB


In [121]:
with open('/Users/kaykay/Downloads/RecognitionEngineProject/recognitionengine/src/ml_determine_stream/df_pmo', 'wb') as fh:  # notice that you need the 'wb' for the dump
    pickle.dump(df3, fh)

In [95]:
df_test = scrape_web_job_description(map_of_streams_tester)

Testing  >>>>>>>
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15


In [96]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        220 non-null    object
 1   Description  220 non-null    object
 2   Stream       220 non-null    object
dtypes: object(3)
memory usage: 5.3+ KB


In [97]:
with open('/Users/kaykay/Downloads/RecognitionEngineProject/recognitionengine/src/ml_determine_stream/df_test', 'wb') as fh:  # notice that you need the 'wb' for the dump
    pickle.dump(df_test, fh)

In [112]:
# SCARPING ITSM
# incorrect name of df
df_test = scrape_web_job_description(map_of_streams_ITSM)

IT Service Management  >>>>>>>
technical support >>>>>
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7


In [113]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        105 non-null    object
 1   Description  105 non-null    object
 2   Stream       105 non-null    object
dtypes: object(3)
memory usage: 2.6+ KB


In [130]:
# Risk
df_risk = scrape_web_job_description(map_of_streams_risk)

Compliance and Risk  >>>>>>>
risk analyst >>>>>
Page: 2
Page: 3
Page: 4
Page: 5


In [131]:
df_risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        75 non-null     object
 1   Description  75 non-null     object
 2   Stream       75 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [137]:
# info_sec
df_info_sec = scrape_web_job_description(map_of_streams_info_security)


Information Security Management  >>>>>>>
information security >>>>>
Page: 2
Page: 3
Page: 4
Page: 5


In [138]:
df_info_sec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        75 non-null     object
 1   Description  75 non-null     object
 2   Stream       75 non-null     object
dtypes: object(3)
memory usage: 1.9+ KB


In [ ]:
# may need more 
# compliance and risk
# RPA
# Cyber Security
# Information Security
# 11 streams >>> 4 needs to be checked